In [1]:
import multiprocessing, torch, re
import os.path as osp
import datasets

from transformers import EncoderDecoderModel, BertTokenizerFast,\
    Seq2SeqTrainer, Seq2SeqTrainingArguments
from tokenization_kobert import KoBertTokenizer
from datasets import load_dataset, Dataset
from evaluate import load as load_metric
from tqdm.auto import tqdm
from time import strftime, time, localtime
from os import listdir
from functools import partial
from typing import List
from zipfile import ZipFile

from utils import rouge, load_dataset

print(torch.__version__)

2.0.0+cu117


In [2]:
# model_path = "skt/kogpt2-base-v2"
# tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path, 
#                                                     bos_token='</s>', eos_token='</s>', unk_token='<unk>',
#                                                     pad_token='<pad>', mask_token='<mask>', max_length=512)
# model = GPT2LMHeadModel.from_pretrained(model_path)
# model = BertForMaskedLM.from_pretrained("skt/kobert-base-v1")
# model = AutoModel.from_pretrained('monologg/distilkobert')
# model = BertForMaskedLM.from_pretrained("monologg/kobert-lm")
# tokenizer = KoBertTokenizer.from_pretrained('monologg/distilkobert')
model = EncoderDecoderModel.from_pretrained("kykim/bertshared-kor-base")
tokenizer = BertTokenizerFast.from_pretrained("kykim/bertshared-kor-base")
model

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [3]:
# sent = "안녕하세요.[MASK][MASK][MASK][MASK][MASK][MASK]"

# collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
# encoded = tokenizer(sent, return_tensors="pt")
# target = tokenizer("안녕하세요", return_tensors='pt')

# out = model(**encoded)
# pipe = FillMaskPipeline(model=model, tokenizer=tokenizer)
# while True:
#     t = model(**encoded)
#     logits:torch.Tensor = t.logits
#     predicted_token = logits[:,-1,:].argmax(dim=1)
#     print(tokenizer.decode(predicted_token))

# print(tokenizer.decode(logits[:,-1,:].argmax(dim=1)))

In [4]:
# pipe("안녕하[MASK][MASK]")

In [5]:
# mask_idxs = encoded['input_ids'] == tokenizer.mask_token_id
# print(mask_idxs)
# print(out.logits.shape)
# masks = out.logits[mask_idxs].argmax(dim=-1)
# print(masks)
# print(tokenizer.decode(masks))
# predicted_token = out.logits[0,-1].argmax(-1)
# tokenizer.decode(predicted_token)

In [6]:
# dataset = load_dataset("HuggingFaceH4/ultrachat_200k")
# dataset = load_dataset("wikipedia", "20220301.en")
# dataset = dataset.remove_columns([col for col in dataset.column_names if col != 'text'])
# dataset = load_dataset("bookcorpus", split="train", streaming=True).with_format('torch')
# dataset = load_dataset("bookcorpus", split="train")
# dataset

In [7]:
# def batch_iterator(batch_size=10000):
#     for i in tqdm(range(0, len(dataset['train']), batch_size)):
#         yield dataset['train'][i:i+batch_size]['text']
# if [_dir for _dir in listdir() if "tokenizer" in _dir] != []:
#     latest_tokenizer_path = sorted([_dir for _dir in listdir() if "tokenizer" in _dir])[-1]
#     tokenizer = AutoTokenizer.from_pretrained(latest_tokenizer_path)
# else:
#     # tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2").train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
#     tokenizer = AutoTokenizer.from_pretrained("skplanet/dialog-koelectra-small-discriminator").train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=32_000)
#     tokenizer.save_pretrained(get_time_dir())

# tokenizer.pad_token = '<pad>'
# tokenizer.eos_token = '</s>'
# tokenizer.bos_token = '</s>'
# tokenizer.unk_token = '<unk>'
# tokenizer.mask_token = '<mask>'
# tokenizer.model_max_length = 32

In [8]:
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

The max length for the tokenizer is: 512


In [9]:
def get_time_dir(): return f"{strftime('%m-%d-%H-%M', localtime(time()))}"

In [10]:
from utils import load_iter_train_test
# dataset = load_dataset(tokenizer=tokenizer)
trainset, testset = load_iter_train_test(tokenizer=tokenizer)

In [11]:
# one = dataset['train'][0]
# one

In [12]:
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

model.config.max_length = 32
model.config.early_stopping = True
# model.config.no_repeat_ngram_size = 1
model.config.length_penalty = 2.
model.config.repetition_penalty = 3.
model.config.num_beams = 10
model.config.vocab_size = model.config.encoder.vocab_size

In [13]:
args = Seq2SeqTrainingArguments(
    output_dir=f"output-{get_time_dir()}",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    evaluation_strategy="steps",
    do_train=True,
    do_eval=True,
    eval_steps=1_000,
    logging_steps=1_000,
    gradient_accumulation_steps=8,
    # num_train_epochs=1,
    weight_decay=.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=2_000,
    fp16=True,
    num_train_epochs=5,
    save_total_limit=1,
    max_steps=100_000
)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    compute_metrics=partial(rouge, tokenizer=tokenizer),
    train_dataset=trainset,
    eval_dataset=testset,
)

In [14]:
# config = AutoConfig.from_pretrained(
#     "gpt2",
#     vocab_size = len(tokenizer),
#     n_ctx = tokenizer.model_max_length,
#     bos_token_id=tokenizer.bos_token_id,
#     eos_token_id=tokenizer.eos_token_id,
# )
# model = GPT2LMHeadModel(config)
# model_size = sum(t.numel() for t in model.parameters())
# print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")
# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [15]:
trainer.train()

c:\Users\sizzf\.conda\envs\torch\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/100000 [00:00<?, ?it/s]

text: 응, 난 엄청 끈처, label: 근처에


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\sizzf\.conda\envs\torch\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
